<a href="https://colab.research.google.com/github/Max-FM/SPRINT-Colombia/blob/main/Downloading_Satellite_Images_From_Google_Earth_Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Downloading Satellite Images From Google Earth Engine 

##Install required packages

In [ ]:
%%capture

!pip install geemap

##Import required packages

In [ ]:
import ee
import folium
import geemap.eefolium as emap

##Authenticate Google Earth Engine

To access the Google Earth Engine API you require an account. To request access, go to [https://signup.earthengine.google.com](https://signup.earthengine.google.com/). You may have to wait up to a day or so to be granted access and it's possible you will not recieve any email communication. To manually check whether you have access, try to log into [https://code.earthengine.google.com](https://code.earthengine.google.com/), or attempt to run the next cell and follow the instructions provided in the output cell.

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=qs_fNPsj7mjxavGEOmSdPMPwpVQR0m2t0uFZHCKb1sI&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/5AElteGN-DH0cdeeMupaG2EJ7M3NJ_TEopru4lugT_5mzMRDIkP4OLQ

Successfully saved authorization token.


##Define Request Function

In [ ]:
def obtain_data(region, start_date, end_date, months_either_side=0, max_cloud_cover=80):
    start_date = ee.Date(start_date)
    start_date = start_date.advance(-months_either_side, 'month')

    end_date = ee.Date(end_date)
    end_date = end_date.advance(months_either_side, 'month')
    
    # Filter input collections by desired date range, region and cloud coverage.
    criteria  = ee.Filter.And(ee.Filter.geometry(region), 
                              ee.Filter.date(start_date, end_date))
    
    Sentinel_2_SR = ee.ImageCollection('COPERNICUS/S2_SR') \
                      .filter(criteria) \
                      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', max_cloud_cover)) \
                      .select(['B4', 'B3', 'B2', 'B8'])

    Sentinel_1_SAR_GRD_C_BAND = ee.ImageCollection("COPERNICUS/S1_GRD") \
                                  .filter(criteria)

    Landsat_8_T1_SR = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR") \
                        .filter(criteria) \
                        .filter(ee.Filter.lt('CLOUD_COVER', max_cloud_cover)) \
                        .select(['B4', 'B3', 'B2', 'B5'])

    Landsat_7_T1_SR = ee.ImageCollection("LANDSAT/LE07/C01/T1_SR") \
                        .filter(criteria) \
                        .filter(ee.Filter.lt('CLOUD_COVER', max_cloud_cover)) \
                        .select(['B3', 'B2', 'B1', 'B4'])

    Landsat_5_T1_SR = ee.ImageCollection("LANDSAT/LT05/C01/T1_SR") \
                        .filter(criteria) \
                        .filter(ee.Filter.lt('CLOUD_COVER', max_cloud_cover)) \
                        .select(['B3', 'B2', 'B1', 'B4'])

    MODIS_16D_NDVI = ee.ImageCollection("MODIS/006/MOD13Q1") \
                       .filter(criteria) \
                       .select('NDVI')

    image_collections = {'Sentinel_2_SR': Sentinel_2_SR,
                         'Sentinel_1_SAR_GRD_C_BAND': Sentinel_1_SAR_GRD_C_BAND,
                         'Landsat_8_T1_SR': Landsat_8_T1_SR,
                         'Landsat_7_T1_SR': Landsat_7_T1_SR,
                         'Landsat_5_T1_SR': Landsat_5_T1_SR,
                         'MODIS_16D_NDVI': MODIS_16D_NDVI}

    return image_collections

##Importing Dates from Spreadsheet and Extracting All Events

In [ ]:
import pandas as pd

extreme_weather_table = pd.read_excel('/content/drive/Shared drives/Colombia SPRINT/Test Districts/Colombia extreme weather in test districts.xlsx')

extreme_weather_table[['Disaster', 'Start Year', 'Month', 'Day', 'End Year', 'End Month', 'End Day']]


,Disaster,Start Year,Month,Day,End Year,End Month,End Day
0,Drought,1998,1.0,NaN,1998,NaN,NaN
1,Earthquake,1999,1.0,25.0,1999,1.0,25.0
2,Flood,1999,1.0,10.0,1999,5.0,19.0
3,Flood,1999,10.0,28.0,1999,12.0,31.0
4,Flood,2000,5.0,18.0,2000,5.0,24.0
5,Wildfire,2001,8.0,NaN,2001,8.0,NaN
6,Drought,2002,NaN,NaN,2003,NaN,NaN
7,Flood,2002,4.0,24.0,2002,4.0,29.0
8,Flood,2003,8.0,NaN,2003,12.0,NaN
9,Flood,2004,1.0,NaN,2004,6.0,28.0


In [ ]:
extreme_weather_table = pd.read_excel('/content/drive/Shared drives/Colombia SPRINT/Test Districts/Colombia extreme weather in test districts.xlsx')

# Filling in empty values for dates.
extreme_weather_table['Day'].fillna(1, inplace=True)
extreme_weather_table['Month'].fillna(1, inplace=True)
extreme_weather_table['End Day'].fillna(1, inplace=True)
extreme_weather_table['End Month'].fillna(1, inplace=True)

# Dealing with duplicate dates
extreme_weather_table.loc[0, 'End Year'] = 1999

# Converting date columns into a single datetime column.
extreme_weather_table['Start_Datetime'] = pd.to_datetime(extreme_weather_table['Start Year'].astype(str) + '-' +
                                          extreme_weather_table['Month'].astype(int).astype(str) + '-' +
                                          extreme_weather_table['Day'].astype(int).astype(str))

extreme_weather_table['End_Datetime'] = pd.to_datetime(extreme_weather_table['End Year'].astype(str) + '-' +
                                        extreme_weather_table['End Month'].astype(int).astype(str) + '-' +
                                        extreme_weather_table['End Day'].astype(int).astype(str))

disaster = 'Flood' # Current options are 'Flood' and 'Drought'.

# Create a tuple of strings containing start and end dates.
start_date_list = list(extreme_weather_table[extreme_weather_table['Disaster'] == disaster]['Start_Datetime'].astype(str))
end_date_list = list(extreme_weather_table[extreme_weather_table['Disaster'] == disaster]['End_Datetime'].astype(str))

date_ranges = list(zip(start_date_list, end_date_list))

display(date_ranges)

[('1999-01-10', '1999-05-19'),
 ('1999-10-28', '1999-12-31'),
 ('2000-05-18', '2000-05-24'),
 ('2002-04-24', '2002-04-29'),
 ('2003-08-01', '2003-12-01'),
 ('2004-01-01', '2004-06-28'),
 ('2005-04-12', '2005-05-07'),
 ('2005-09-15', '2005-11-17'),
 ('2006-01-01', '2006-04-27'),
 ('2007-10-20', '2007-10-26'),
 ('2008-01-01', '2008-05-19'),
 ('2008-11-16', '2009-01-12'),
 ('2010-10-30', '2011-01-12'),
 ('2011-02-10', '2011-06-05'),
 ('2011-09-01', '2011-12-31'),
 ('2012-03-15', '2012-05-14'),
 ('2013-09-15', '2013-12-01'),
 ('2017-03-17', '2017-05-16'),
 ('2017-12-01', '2018-01-07'),
 ('2019-02-20', '2019-02-26'),
 ('2020-06-10', '2020-06-10')]

##Downloading Imaging for All Events

In [ ]:
test_districts = {'Dosquebradas': ee.FeatureCollection('users/maxfoxley-marrable/Dosquebradas'),
                  'Corpo Versailles': ee.FeatureCollection('users/maxfoxley-marrable/Versailles')}

scale_dict = {'Sentinel_2_SR': 10,
              'Sentinel_1_SAR_GRD_C_BAND': 10,
              'Landsat_8_T1_SR': 30,
              'Landsat_7_T1_SR': 30,
              'Landsat_5_T1_SR': 30,
              'MODIS_16D_NDVI': 250}

bands_dict = {'Sentinel_2_SR': ['B4', 'B3', 'B2'], 
            #   'Sentinel_1_SAR_GRD_C_BAND': 
              'Landsat_8_T1_SR': ['B4', 'B3', 'B2'],
              'Landsat_7_T1_SR': ['B3', 'B2', 'B1'],
              'Landsat_5_T1_SR': ['B3', 'B2', 'B1'],
              'MODIS_16D_NDVI': 'NDVI'} 

if disaster == 'Drought':
    # Probably only want MODIS 16D Imagery for droughts.
    desired_collections = ['MODIS_16D_NDVI'] 
else:
    # e.g ['Sentinel_2_SR', 'Landsat_8_T1_SR', etc]. Selects all if empty/None.
    desired_collections = ['Sentinel_1_SAR_GRD_C_BAND'] #None 

In [ ]:
for district_name, district_geom in test_districts.items():
    drive_dir = f'/content/drive/Shared drives/Colombia SPRINT/Test Districts/{district_name}/Extreme Weather Event Imagery/{disaster}s/'
    %cd '{drive_dir}'
    for start_date, end_date in date_ranges:

        image_collections = obtain_data(district_geom, 
                                        start_date, 
                                        end_date,
                                        months_either_side=2,
                                        max_cloud_cover=80)

        if desired_collections:
            image_collections = {collection: image_collections[collection] \
                                 for collection in desired_collections}

        out_dir = f'{start_date}'
        print(out_dir)
        !mkdir '{out_dir}'

        for collection_name, collection in image_collections.items():

            if collection.size().getInfo() == 0:
                print('No images in collection, skipping.')
                continue
    
            collection_dir = f'{out_dir}/{collection_name}'
            print(collection_dir)
            !mkdir '{collection_dir}'

            emap.ee_export_image_collection(collection, 
                                            collection_dir,
                                            crs='EPSG:4326',
                                            scale=scale_dict[collection_name],
                                            region=district_geom.geometry().bounds())


/content/drive/Shared drives/Colombia SPRINT/Test Districts/Dosquebradas/Extreme Weather Event Imagery/Floods
1999-01-10
mkdir: cannot create directory ‘1999-01-10’: File exists
No images in collection, skipping.
1999-10-28
mkdir: cannot create directory ‘1999-10-28’: File exists
No images in collection, skipping.
2000-05-18
mkdir: cannot create directory ‘2000-05-18’: File exists
No images in collection, skipping.
2002-04-24
mkdir: cannot create directory ‘2002-04-24’: File exists
No images in collection, skipping.
2003-08-01
mkdir: cannot create directory ‘2003-08-01’: File exists
No images in collection, skipping.
2004-01-01
mkdir: cannot create directory ‘2004-01-01’: File exists
No images in collection, skipping.
2005-04-12
mkdir: cannot create directory ‘2005-04-12’: File exists
No images in collection, skipping.
2005-09-15
mkdir: cannot create directory ‘2005-09-15’: File exists
No images in collection, skipping.
2006-01-01
mkdir: cannot create directory ‘2006-01-01’: File exist

##Create Timelapse GIFs of Each Image Collection

In [ ]:
if disaster == 'Drought':
    desired_collections = ['MODIS_16D_NDVI'] # Probably only want MODIS 16D Imagery for droughts.
else:
    desired_collections = None # e.g ['Sentinel_2_SR', 'Landsat_8_T1_SR', etc]. Selects all if empty.

for district_name, district_geom in test_districts.items():
    drive_dir = f'/content/drive/Shared drives/Colombia SPRINT/Test Districts/{district_name}/Extreme Weather Event Imagery/{disaster}s/'
    %cd '{drive_dir}'
    for start_date, end_date in date_ranges:

        image_collections = obtain_data(district_geom, 
                                        start_date, 
                                        end_date,
                                        months_either_side=2,
                                        max_cloud_cover=80)

        if desired_collections:
            image_collections = {collection: image_collections[collection] \
                                 for collection in desired_collections}

        out_dir = f'{start_date}'
        print(out_dir)

        for collection_name, collection in image_collections.items():

            if collection.size().getInfo() == 0:
                print('No images in collection, skipping.')
                continue
            
            if collection_name == 'MODIS_16D_NDVI':
                gif_params = {'bands': bands_dict[collection_name],
                              'dimensions': 1000,
                              'crs': 'EPSG:4326',
                              'framesPerSecond': 1,
                              'min': 0,
                              'max': 8000,
                              'palette': ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
                                          '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
                                          '012E01', '011D01', '011301'],
                              'region': district_geom.geometry().bounds()}
            
            else:
                gif_params = {'bands': bands_dict[collection_name],
                              'dimensions': 1000,
                              'crs': 'EPSG:4326',
                              'framesPerSecond': 1,
                              'min': 0, 
                              'max': 3000,
                              'region': district_geom.geometry().bounds()}

            gif_path = f'{out_dir}/{collection_name}_{start_date}.gif'

            emap.download_ee_video(collection, 
                                   gif_params,
                                   gif_path)
            
            # geemap.add_text_to_gif(in_gif, out_gif, xy=('5%', '5%'), text_sequence=f'{collection_name}_{date}', font_size=30, font_color='#0000ff', duration=100)


/content/drive/Shared drives/Colombia SPRINT/Test Districts/Dosquebradas/Extreme Weather Event Imagery/Droughts
1998-01-01
No images in collection, skipping.
2002-01-01
Generating URL...
Please wait ...
The GIF image has been saved to: /content/drive/Shared drives/Colombia SPRINT/Test Districts/Dosquebradas/Extreme Weather Event Imagery/Droughts/2002-01-01/MODIS_16D_NDVI_2002-01-01.gif
2004-01-01
Generating URL...
Please wait ...
The GIF image has been saved to: /content/drive/Shared drives/Colombia SPRINT/Test Districts/Dosquebradas/Extreme Weather Event Imagery/Droughts/2004-01-01/MODIS_16D_NDVI_2004-01-01.gif
2006-01-01
Generating URL...
Please wait ...
The GIF image has been saved to: /content/drive/Shared drives/Colombia SPRINT/Test Districts/Dosquebradas/Extreme Weather Event Imagery/Droughts/2006-01-01/MODIS_16D_NDVI_2006-01-01.gif
2015-08-01
Generating URL...
Please wait ...
The GIF image has been saved to: /content/drive/Shared drives/Colombia SPRINT/Test Districts/Dosquebrada